Import relevant python packages... 

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Import the analysis package.

In [ ]:
import cosmicfish as cf 

Specify the values of your fiducial cosmology and other physical constants.

In [ ]:
fid = {"A_s" : 2.2321e-9, 
        "n_s" : 0.96659,
        "omega_b" : 0.02226,
        "omega_cdm" : 0.11271,
        "tau_reio" : 0.059888,
        "h" : 0.70148} #Not necessary, but helps neatness later on

A_s = 2.2321e-9
n_s = 0.96659
omega_b = 0.02226
omega_cdm = 0.11271
tau_reio = 0.059888
h = 0.70148

c = 2.9979e8 #Units of m/s
H = 1000. * 100. * fid['h'] #Units of m/s/Mpc
kp = 0.05 * fid['h'] #Units of 1/Mpc

Specify points of interest to accumulate power spectra at... 

In [ ]:
z_table = np.arange(0.7, 2.1, 0.1)
m_chi_table = np.logspace(-1.0, 1.0, num=21, endpoint=True)

Determine derived quantities... Here, we assume the following relations: 

$$ \omega_\chi = \Big(\frac{T_\chi}{1.95 \textrm{[K]} }\Big)^3 \Big( \frac{m_\chi}{94 \textrm{[eV]}}\Big) $$

$$ \omega_m = \omega_b + \omega_{cdm} + \omega_\chi $$

$$ \omega_\lambda = h^2 - \omega_m$$

$$ V_{eff} = \Big(\frac{4 \pi}{3}\Big)\Big(\frac{c}{H}\Big)^3\Big( \int_{0.7}^{2.0} \frac{h}{\sqrt{\omega_m (1+z)^3 + \omega_\lambda}}  ~dz\Big)^3 $$

In [ ]:
T_chi = 0.5 #CAUTION! USING A SINGLE T VALUE TO CALCULATE THIS BLOCK!
omega_chi_table = np.power((T_chi / 1.95), 3.) * (m_chi_table / 94.)
omega_m_table = fid["omega_b"] + fid["omega_cdm"] + omega_chi_table
omega_lambda_table = np.power(h, 2.) - omega_m_table
v_eff_table = cf.v_effective_table_generator(z_table, omega_m_table, omega_lambda_table, c, h, H)
k_table = cf.k_table_generator(v_eff_table, h, 0.1, 100)

At this point, we have $\omega_m$, $\omega_{\lambda}$, $\omega_{\chi}$, $V_{eff}$ for several values of $m_\chi$ assuming a particular value of $T_\chi$. For transparency, let's show what these all look like... 

In [ ]:
fig, ax = plt.subplots(1,4, figsize=(20, 5))
ax[0].plot(m_chi_table, omega_chi_table)
ax[0].set_title(r'$\omega_\chi$')
ax[0].set_xlabel(r'$m_\chi$ [eV]')
ax[1].plot(m_chi_table, omega_lambda_table, label=r'$\omega_\lambda$')
ax[1].set_title(r'$\omega_\lambda$')
ax[1].set_xlabel(r'$m_\chi$ [eV]')
ax[2].plot(m_chi_table, omega_m_table, label=r'$\omega_m$')
ax[2].set_title(r'$\omega_m$')
ax[2].set_xlabel(r'$m_\chi$ [eV]')
ax[3].plot(m_chi_table, v_eff_table)
ax[3].set_title(r'$V_{eff}$ [Mpc$^3$]')
ax[3].set_xlabel(r'$m_\chi$ [eV]')
plt.show()

These all behave in an obvious way. Notice also that our k_table, which spans 100 points at each choice of $m_\chi$, has the first few values of k given by: 

In [ ]:
pd.DataFrame(k_table).head()

Define path where all data runs are stored. Anytime we require spectrum data from CLASS, the code will first check whether such data has already been calculated. If so, it will just import that data rather than re-run CLASS. We also specify a path to the CLASS install directory. 

In [ ]:
datastore = cf.correct_path("~/Desktop/test/data/")
classpath = cf.correct_path("~/Desktop/test/lib/class/")

If you'd like to test convergence, specify the details of your convergence tests... 

In [ ]:
test_convergence = True
non_chi_vary_relative = [1.01, 1.02, 1.03, 1.04, 1.05]
t_chi_vary_absolute = [0.50, 1.00, 1.50, 2.00, 2.50, 10.00]

omega_b_gen_data = non_chi_vary_relative * omega_b
omega_cdm_gen_data = non_chi_vary_relative * omega_cdm
tau_reio_gen_data = non_chi_vary_relative * tau_reio
h_gen_data = non_chi_vary_relative * h
t_chi_gen_data = t_chi_vary_absolute

Import and/or generate relevant datasets... 

Interpolate and calculate numerical derivatives...

See how things look... 

In [ ]:
for _, e, _ in os.walk("/Users/nicholasdeporzio/Desktop/test/data/"): 
    print(e)

In [ ]:
for e in next(os.walk("/Users/nicholasdeporzio/Desktop/test/data/"))[1]: 
    print(int(e))

In [ ]:
s1 = cf.spectrum("~/Desktop/test/data/1/")

In [ ]:
s1.rawdata.head()

In [ ]:
help(cf.generate_data)

In [ ]:
cf.generate_data(fid, "~/Desktop/test/lib/class", "~/Desktop/test/data", n_s=0.9)

In [ ]:
os.getcwd()